# YOLO GPU Test & Image Analysis

Test YOLO object detection with GPU acceleration on your test images.

In [ ]:
import torch
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Verify GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Load YOLO model (will auto-download on first run)
# Options: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x (nano to extra-large)
# Start with 'n' (nano) for quick testing
model = YOLO('yolov8n.pt')

# Verify model is on GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(f"Model loaded on: {device}")

In [ ]:
# Test on your fixture images
test_images = [
    "../tests/fixtures/2024-10-11_12-19-49.jpg",
    "../tests/fixtures/2024-10-11_12-19-50.dng",
    "../tests/fixtures/2024-10-11_12-19-51.dng"
]

for img_path in test_images:
    if not Path(img_path).exists():
        print(f"Skipping {img_path} - file not found")
        continue
    
    print(f"\n{'='*60}")
    print(f"Processing: {Path(img_path).name}")
    print('='*60)
    
    # Run inference
    results = model(img_path, conf=0.25, device=device)
    
    # Display results
    result = results[0]
    print(f"\nDetected {len(result.boxes)} objects:")
    
    for box in result.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        name = result.names[cls]
        print(f"  - {name}: {conf:.2%} confidence")
    
    # Plot with bounding boxes
    plt.figure(figsize=(12, 8))
    img_with_boxes = result.plot()
    plt.imshow(img_with_boxes)
    plt.axis('off')
    plt.title(f"{Path(img_path).name} - {len(result.boxes)} objects detected")
    plt.tight_layout()
    plt.show()

In [ ]:
# Performance benchmark
import time

test_img = "../tests/fixtures/2024-10-11_12-19-49.jpg"
n_runs = 10

times = []
for i in range(n_runs):
    start = time.time()
    _ = model(test_img, device=device, verbose=False)
    times.append(time.time() - start)

print(f"\nGPU Performance Benchmark ({n_runs} runs):")
print(f"  Mean: {np.mean(times)*1000:.1f} ms")
print(f"  Std:  {np.std(times)*1000:.1f} ms")
print(f"  Min:  {np.min(times)*1000:.1f} ms")
print(f"  Max:  {np.max(times)*1000:.1f} ms")

## Custom Object Detection

YOLO can detect specific classes you care about.
Filter for nature/plant-related objects:

In [ ]:
# Nature-related classes in COCO dataset
nature_classes = {
    'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 
    'zebra', 'giraffe', 'potted plant', 'flower', 'tree', 'mushroom'
}

test_img = "../tests/fixtures/2024-10-11_12-19-49.jpg"
results = model(test_img, conf=0.25, device=device)
result = results[0]

print("Nature-related detections:")
found_nature = False
for box in result.boxes:
    cls = int(box.cls[0])
    name = result.names[cls]
    if name.lower() in nature_classes:
        conf = float(box.conf[0])
        print(f"  - {name}: {conf:.2%}")
        found_nature = True

if not found_nature:
    print("  (No nature objects detected)")

## Integration Ideas

**For Your Project:**
- Auto-tag images based on detected objects
- Filter images containing specific subjects (plants, animals, people)
- Quality scoring based on composition (rule of thirds, subject placement)
- Scene understanding to complement your taxonomy links